# Study Case: Simple ETL Process from Scratch

**Extract**

*  Scrape data tablet dari https://webscraper.io/test-sites/e-commerce/static/computers/tablets?page=1
*  Ambil data dari page 1 s/d 4
*  Ambil title, price, description, & jumlah review
*  Masukkan data tsb ke pandas dataframe

**Transform**

*  Construct id column
*  Pastikan id column tidak ada duplikat
*  Convert price menjadi decimal
*  Convert jml review menjadi integer (ambil angkanya saja)

**Load**

*  Buat table dgn nama: tablet_<nama>, dgn kolom id sbg primary key
*  Insert dataframe ke dalam table tsb (bisa menggunakan SQL dibungkus python, atau pandas.to_sql()

## Preparation

Install library yang diperlukan, seperti `SQLAlchemy` dan `PyMySQL`.

In [1]:
!pip install sqlalchemy

In [2]:
!pip install pymysql

     |████████████████████████████████| 43 kB 1.0 MB/s 


In [3]:
!pip install python-dotenv

Import library yang dibutuhkan selama proses ETL (Extract, Transform, Load).

In [4]:
import os
import uuid
import requests
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import types, create_engine
from sqlalchemy.engine.url import URL
from google.colab import files
from dotenv import load_dotenv
from dotenv import dotenv_values
load_dotenv()

True

Jangan lupa untuk upload `env.example` yang telah diberikan sebelum menjalankan semua proses.

In [5]:
uploaded = files.upload()

Saving env.example to env.example


Setelah itu, bisa jalankan `os.getenv("NAMA_VARIABLE")` untuk mendapatkan suatu variabel di dalam `.env`

In [6]:
# Rename file env menjadi .env
os.rename("env.example", ".env")

In [7]:
# Load isi dalam .env ke dalam config
config = dotenv_values(".env")

## Extract Process (E)

### Mendapatkan konten HTML

URL yang akan dilakukan scraping adalah `https://webscraper.io/test-sites/e-commerce/static/computers/tablets?page={page}` dimana variabel `page` bisa diubah sesuai kebutuhan, seperti di study case dari page 1 sampai 4.

In [8]:
# Fungsi scraping_web dengan parameter url dan mengembalikan konten dari web
def scraping_web(url):
  response = requests.get(url)
  return response.text

In [9]:
url = "https://webscraper.io/test-sites/e-commerce/static/computers/tablets?page="
html_contents = ""

# Mendapatkan semua isi dari web
for page in range(1, 5):
  html = scraping_web(url + f"{page}")
  html_contents += html

### Parsing konten untuk mendapatkan data

In [10]:
# Parsing html_contents ke dalam bentuk BeautifulSoup
parsed_contents = BeautifulSoup(html_contents, "html.parser")
print(type(parsed_contents))

<class 'bs4.BeautifulSoup'>


Mendapatkan title dari `parsed_contents`

In [11]:
# Mendapatkan semua tag a dg class title
titles = parsed_contents.find_all("a", class_="title")

# Ambil hanya text di dalam tag a tersebut
list_title = []
for title in titles:
  list_title.append(title.text)

print(list_title)

['Lenovo IdeaTab', 'Acer Iconia', 'Asus MeMO Pad', 'Amazon Kindle', 'iPad Mini Retina', 'IdeaTab A3500L', 'Galaxy Tab', 'IdeaTab A3500-H', 'Galaxy Tab 3', 'Galaxy Note', 'MeMO Pad 7', 'Galaxy Tab 4', 'Galaxy Tab 3', 'Memo Pad HD 7', 'Galaxy Note 10.1', 'MeMo PAD FHD 10', 'IdeaTab S5000', 'Iconia B1-730HD', 'IdeaTab A8-50', 'Galaxy Note', 'Apple iPad Air']


Mendapatkan price dari `parsed_contents`

In [12]:
# Mendapatkan semua tag h4 dg class price
prices = parsed_contents.find_all("h4", class_="price")

# Ambil hanya text di dalam tag h4 tersebut
list_price = []
for price in prices:
  list_price.append(price.text)

print(list_price)

['$69.99', '$96.99', '$102.99', '$103.99', '$537.99', '$88.99', '$251.99', '$148.99', '$97.99', '$489.99', '$130.99', '$233.99', '$107.99', '$101.99', '$587.99', '$320.99', '$172.99', '$99.99', '$121.99', '$399.99', '$603.99']


Mendapatkan description dari `parsed_contents`

In [13]:
# Mendapatkan semua tag p dg class description
descriptions = parsed_contents.find_all("p", class_="description")

# Ambil hanya text di dalam tag p tersebut
list_description = []
for description in descriptions:
  list_description.append(description.text)

print(list_description)

['7" screen, Android', '7" screen, Android, 16GB', '7" screen, Android, 8GB', '6" screen, wifi', 'Wi-Fi + Cellular, 32GB, Silver', 'Black, 7" IPS, Quad-Core 1.2GHz, 8GB, Android 4.2', '16GB, White', 'Blue, 7" IPS, Quad-Core 1.3GHz, 8GB, 3G, Android 4.2', '7", 8GB, Wi-Fi, Android 4.2, White', '12.2", 32GB, WiFi, Android 4.4, White', 'White, 7", Atom 1.2GHz, 8GB, Android 4.4', 'LTE (SM-T235), Quad-Core 1.2GHz, 8GB, Black', '7", 8GB, Wi-Fi, Android 4.2, Yellow', 'IPS, Dual-Core 1.2GHz, 8GB, Android 4.3', '10.1", 32GB, Black', 'White, 10.1" IPS, 1.6GHz, 2GB, 16GB, Android 4.2', 'Silver, 7" IPS, Quad-Core 1.2Ghz, 16GB, 3G, Android 4.2', 'Black, 7", 1.6GHz Dual-Core, 8GB, Android 4.4', 'Blue, 8" IPS, Quad-Core 1.3GHz, 16GB, Android 4.2', '10.1", 3G, Android 4.0, Garnet Red', 'Wi-Fi, 64GB, Silver']


Mendapatkan jumlah review dari `parsed_contents`

In [14]:
# Mendapatkan semua tag p dg class pull-right
num_reviews = parsed_contents.find_all("p", class_="pull-right")

# Ambil hanya text di dalam tag p tersebut
list_num_review = []
for num_review in num_reviews:
  list_num_review.append(num_review.text)

print(list_num_review)

['7 reviews', '7 reviews', '14 reviews', '3 reviews', '8 reviews', '7 reviews', '14 reviews', '9 reviews', '2 reviews', '9 reviews', '11 reviews', '1 reviews', '14 reviews', '10 reviews', '6 reviews', '7 reviews', '8 reviews', '1 reviews', '13 reviews', '12 reviews', '7 reviews']


### Membuat DataFrame untuk manipulasi data

In [15]:
# Membuat dataframe baru dengan memasukkan data dari 4 list di atas
tablets_df = pd.DataFrame({
    "title": list_title,
    "description": list_description,
    "price": list_price,
    "num_review": list_num_review
})

print("Shape of tablet_df:", tablets_df.shape)

Shape of tablet_df: (21, 4)


In [16]:
tablets_df.head()

,title,description,price,num_review
0,Lenovo IdeaTab,"7"" screen, Android",$69.99,7 reviews
1,Acer Iconia,"7"" screen, Android, 16GB",$96.99,7 reviews
2,Asus MeMO Pad,"7"" screen, Android, 8GB",$102.99,14 reviews
3,Amazon Kindle,"6"" screen, wifi",$103.99,3 reviews
4,iPad Mini Retina,"Wi-Fi + Cellular, 32GB, Silver",$537.99,8 reviews


## Transform Process (T)

### Mengecek tipe data dan duplikasi data

In [17]:
tablets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        21 non-null     object
 1   description  21 non-null     object
 2   price        21 non-null     object
 3   num_review   21 non-null     object
dtypes: object(4)
memory usage: 800.0+ bytes


In [18]:
# Cek apakah ada nilai > 1 (duplikasi)
tablets_df.value_counts(ascending=False)

title             description                                              price    num_review
iPad Mini Retina  Wi-Fi + Cellular, 32GB, Silver                           $537.99  8 reviews     1
Galaxy Tab 3      7", 8GB, Wi-Fi, Android 4.2, Yellow                      $107.99  14 reviews    1
Amazon Kindle     6" screen, wifi                                          $103.99  3 reviews     1
Apple iPad Air    Wi-Fi, 64GB, Silver                                      $603.99  7 reviews     1
Asus MeMO Pad     7" screen, Android, 8GB                                  $102.99  14 reviews    1
Galaxy Note       10.1", 3G, Android 4.0, Garnet Red                       $399.99  12 reviews    1
                  12.2", 32GB, WiFi, Android 4.4, White                    $489.99  9 reviews     1
Galaxy Note 10.1  10.1", 32GB, Black                                       $587.99  6 reviews     1
Galaxy Tab        16GB, White                                              $251.99  14 reviews    1
Galax

### Membuat kolom `id`

In [19]:
# Membuat kolom id dengan bantuan UUID
# Convert UUID dalam bentuk 32-character hexadecimal string
uuids = [uuid.uuid4().hex for i in range(0, len(tablets_df))]

tablets_df["id"] = pd.Series(uuids)
tablets_df.head()

,title,description,price,num_review,id
0,Lenovo IdeaTab,"7"" screen, Android",$69.99,7 reviews,1db6a2f811034c7f8383fc2ec214b39a
1,Acer Iconia,"7"" screen, Android, 16GB",$96.99,7 reviews,788ee41d349e4d89ae8cfd7461d7d795
2,Asus MeMO Pad,"7"" screen, Android, 8GB",$102.99,14 reviews,abefeeb8080d45429cba452a82240f5f
3,Amazon Kindle,"6"" screen, wifi",$103.99,3 reviews,187eb23b59744d0d90ffbd1d628a8be2
4,iPad Mini Retina,"Wi-Fi + Cellular, 32GB, Silver",$537.99,8 reviews,80ec7a0072b44d21922337bafdeb4f6a


Memastikan `id` yang telah dibentuk unik

In [20]:
tablets_df["id"].value_counts(ascending=False)

408d3a531d9741588fbb935fdc64507c    1
c0c0da33925a4b4d9d5862974fb87f33    1
eed756aea01a49c79975a1518cbe8654    1
e659f9f04b8446188b44c776db034087    1
163d74164dd445319732a779a1a046ea    1
87af92869e154c5db7d2b041ff27c5f4    1
0a34347500af419a898943115adcbf79    1
f1c1f2a76b984fcd8787ba7c5754ad98    1
3877f217a5854b1db331c41071c757dd    1
d8896c2002134970a6a9282e97a18262    1
e8ef4d5360604067a1b8c899d519e1ef    1
a4e95bd20ddf4125b495e4d0c852086a    1
187eb23b59744d0d90ffbd1d628a8be2    1
3598473fa33046d88a0276adcb0853ea    1
c03643db58fe419f9ebf00c3a600aa24    1
80ec7a0072b44d21922337bafdeb4f6a    1
b6df8d05ac7f4e319d716e1af82efab4    1
1db6a2f811034c7f8383fc2ec214b39a    1
788ee41d349e4d89ae8cfd7461d7d795    1
abefeeb8080d45429cba452a82240f5f    1
37cd5bf410ce4b7a8067cf5598fd35ad    1
Name: id, dtype: int64

Sedikit re-arrange kolom df dengan meletakkan kolom `id` di depan.

In [21]:
# Arrange kolomnya supaya id ada di depan
columns = ["id", "title", "description", "price", "num_review"]

tablets_df = tablets_df[columns]
tablets_df.head()

,id,title,description,price,num_review
0,1db6a2f811034c7f8383fc2ec214b39a,Lenovo IdeaTab,"7"" screen, Android",$69.99,7 reviews
1,788ee41d349e4d89ae8cfd7461d7d795,Acer Iconia,"7"" screen, Android, 16GB",$96.99,7 reviews
2,abefeeb8080d45429cba452a82240f5f,Asus MeMO Pad,"7"" screen, Android, 8GB",$102.99,14 reviews
3,187eb23b59744d0d90ffbd1d628a8be2,Amazon Kindle,"6"" screen, wifi",$103.99,3 reviews
4,80ec7a0072b44d21922337bafdeb4f6a,iPad Mini Retina,"Wi-Fi + Cellular, 32GB, Silver",$537.99,8 reviews


### Mengkonversi kolom `price` menjadi desimal

In [22]:
# Cek tipe data untuk price masih object
tablets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           21 non-null     object
 1   title        21 non-null     object
 2   description  21 non-null     object
 3   price        21 non-null     object
 4   num_review   21 non-null     object
dtypes: object(5)
memory usage: 968.0+ bytes


In [23]:
def convert_price(data):
  data_replace = str(data).replace("$", "")
  return float(data_replace)

tablets_df["price"] = tablets_df["price"].apply(lambda x: convert_price(x))
tablets_df.head()

,id,title,description,price,num_review
0,1db6a2f811034c7f8383fc2ec214b39a,Lenovo IdeaTab,"7"" screen, Android",69.99,7 reviews
1,788ee41d349e4d89ae8cfd7461d7d795,Acer Iconia,"7"" screen, Android, 16GB",96.99,7 reviews
2,abefeeb8080d45429cba452a82240f5f,Asus MeMO Pad,"7"" screen, Android, 8GB",102.99,14 reviews
3,187eb23b59744d0d90ffbd1d628a8be2,Amazon Kindle,"6"" screen, wifi",103.99,3 reviews
4,80ec7a0072b44d21922337bafdeb4f6a,iPad Mini Retina,"Wi-Fi + Cellular, 32GB, Silver",537.99,8 reviews


In [24]:
# Cek tipe data dari price sudah berubah menjadi desimal
tablets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           21 non-null     object 
 1   title        21 non-null     object 
 2   description  21 non-null     object 
 3   price        21 non-null     float64
 4   num_review   21 non-null     object 
dtypes: float64(1), object(4)
memory usage: 968.0+ bytes


### Mengkonversi kolom `num_review` menjadi integer (ambil angka saja)

In [25]:
# Cek tipe data untuk num_review masih object
tablets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           21 non-null     object 
 1   title        21 non-null     object 
 2   description  21 non-null     object 
 3   price        21 non-null     float64
 4   num_review   21 non-null     object 
dtypes: float64(1), object(4)
memory usage: 968.0+ bytes


In [26]:
def convert_num_review(data):
  data_split = str(data).split(" ")
  num_only = data_split[0]
  return int(num_only)

tablets_df["num_review"] = tablets_df["num_review"].apply(lambda x: convert_num_review(x))
tablets_df.head()

,id,title,description,price,num_review
0,1db6a2f811034c7f8383fc2ec214b39a,Lenovo IdeaTab,"7"" screen, Android",69.99,7
1,788ee41d349e4d89ae8cfd7461d7d795,Acer Iconia,"7"" screen, Android, 16GB",96.99,7
2,abefeeb8080d45429cba452a82240f5f,Asus MeMO Pad,"7"" screen, Android, 8GB",102.99,14
3,187eb23b59744d0d90ffbd1d628a8be2,Amazon Kindle,"6"" screen, wifi",103.99,3
4,80ec7a0072b44d21922337bafdeb4f6a,iPad Mini Retina,"Wi-Fi + Cellular, 32GB, Silver",537.99,8


In [27]:
# Cek tipe data dari num_review sudah berubah menjadi integer
tablets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           21 non-null     object 
 1   title        21 non-null     object 
 2   description  21 non-null     object 
 3   price        21 non-null     float64
 4   num_review   21 non-null     int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 968.0+ bytes


## Load Process (L)

### Membuat koneksi ke DB

In [28]:
# Setup engine
engine = create_engine(
  URL.create(
    drivername=config["DB_DRIVERNAME"],
    username=config["DB_USERNAME"],
    password=config["DB_PASSWORD"],
    host=config["DB_HOST"],
    port=config["DB_PORT"],
    database=config["DB_DATABASE"]
  )
)

In [29]:
# Konek ke engine
conn = engine.connect()

### Membuat table `tablet_` dengan kolom `id` sebagai primary key

In [30]:
# sql query
query = f"""
  CREATE TABLE IF NOT EXISTS {config["TABLE_NAME"]} (
    id VARCHAR(255) PRIMARY KEY NOT NULL,
    title VARCHAR(255),
    description TEXT,
    price DECIMAL(10,2),
    num_review INT(4)
  )
"""

# execute sql query
query_result = conn.execute(query)

Cek dengan menggunakan [DBeaver](https://dbeaver.io) tabel tersebut apakah sudah berhasil dibuat.

### Insert dataframe ke dalam table tsb menggunakan `pandas.to_sql()`

In [31]:
# Insert data
tablets_df.to_sql(config["TABLE_NAME"], engine, if_exists="replace", index=False, chunksize=5, dtype={
    "id": types.VARCHAR(length=255),
    "title": types.VARCHAR(length=255),
    "description": types.TEXT,
    "price": types.DECIMAL(10,2),
    "num_review": types.INTEGER
})

Cek dengan menggunakan [DBeaver](https://dbeaver.io) tabel tersebut apakah sudah terisi datanya.

### Membaca data yang telah disimpan ke dalam DB

Membaca data dengan menggunakan `pd.read_sql()`

In [32]:
# Membuat query
read_query = "SELECT * FROM " + config["TABLE_NAME"]
# Execute query
read_sql_df = pd.read_sql(read_query, engine)

# shape
print(read_sql_df.shape)

read_sql_df.head()

(21, 5)


,id,title,description,price,num_review
0,1db6a2f811034c7f8383fc2ec214b39a,Lenovo IdeaTab,"7"" screen, Android",69.99,7
1,788ee41d349e4d89ae8cfd7461d7d795,Acer Iconia,"7"" screen, Android, 16GB",96.99,7
2,abefeeb8080d45429cba452a82240f5f,Asus MeMO Pad,"7"" screen, Android, 8GB",102.99,14
3,187eb23b59744d0d90ffbd1d628a8be2,Amazon Kindle,"6"" screen, wifi",103.99,3
4,80ec7a0072b44d21922337bafdeb4f6a,iPad Mini Retina,"Wi-Fi + Cellular, 32GB, Silver",537.99,8
